# Temporal Baseline

The temporal baseline selects the pollutant concentration at the same location and time period of the fire event for the years that the fire event did not take place.   

In [1]:
import os, sys, glob

# adds the package path to the Python path to make sure all the local imports work fine 
if os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))) not in sys.path:
    sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))

# local imports
from wp4.constants import POLLUTANTS, DATA_DIR_CAMS, DB_HOST, DB_NAME, DB_USER, DB_PASS
from wp4.baseline.temporal import get_temporal_baseline

# import remaining packages needed for the script
import xarray as xr
import pandas as pd
import psycopg2 
from datetime import datetime, timedelta

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

## Initiate connection and load fire events from the Postgres database

In [2]:
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST)

query = """
    SELECT id, datetime, ST_X(geometry), ST_Y(geometry), source, location, reference, type, info
    FROM public.all_fire_events
    WHERE reference = 'Aqua' OR reference = 'Terra'
"""

df_fire_events = pd.read_sql_query(query,con=conn).rename(columns = {'st_x':'longitude', 'st_y':'latitude'})

conn.close()

In [ ]:
def create_plot(df_baseline, name_pollutant, file_name=None):
    """
    Creates a plot of the temporal baseline concentration range and the pollutant concentration 
    at the location of the Fire event itself
    """
    fig = go.Figure()  # init a plotly figure

    # Update xaxis properties
    fig.update_xaxes(title_text="Date")
    
    # Update yaxis properties
    fig.update_yaxes(title_text=f"{name_pollutant} Concentration µg m<sup>-3</sup>")
    
    fig.update_layout(
        template=pio.templates["seaborn"],
        autosize=False,
        width=1000,
        height=500
    )  
    
    fig.update_layout(legend=dict(  # position the legend
        yanchor="top",
        y=-0.2,
        xanchor="left",
        x=0.01)
    )
    
    colors = {
        'Q1':"#29bf12",
        'Q3':"#29bf12",
        'MED':"#48cae4",
        'FE':"#f21b3f"
    }
                      
    fig.add_trace(go.Scatter(  # add the baseline values
                x=df_baseline['time'],
                y=df_baseline['temporal_baseline_lower_quartile'].rolling(window=3, min_periods=1).mean(),
                mode='lines',
                name='25th percentile',
                line={'color': colors['Q1']}),
            )
            
    fig.add_trace(go.Scatter(  # add the baseline values
                x=df_baseline['time'],
                y=df_baseline['temporal_baseline_upper_quartile'].rolling(window=3, min_periods=1).mean(),
                fill='tonexty', # fill area between trace0 and trace1
                mode='lines',
                name='75th percentile',
                line={'color': colors['Q1']}),
            )
            
    fig.add_trace(go.Scatter(  # add the baseline values
                x=df_baseline['time'],
                y=df_baseline['temporal_baseline_median'].rolling(window=3, min_periods=1).mean(),
                mode='lines',
                name='Median',
                line={'color': colors['MED']}),
            )
    
    fig.add_trace(go.Scatter(  # add the CAMS analysis concentration data closest to the fireevent
                x=df_baseline['time'],
                y=df_baseline['fire_event'].rolling(window=3, min_periods=1).mean(),
                mode='lines',
                name='CAMS Regional Analysis cell closest to fire event',
                line={'color': colors['FE']}),
            )
    
    if file_name is not None:
        pass
    
    return fig



## Calculate temporal baseline, plot it along with the pollutant concentration during the fire event. 

In [ ]:
pollutant = 'PM25'  # Can be any of 'CO', 'O3', 'NO', 'NO2', 'PM25', 'PM10', 'SO2'
name_pollutant = POLLUTANTS[pollutant]['FULL_NAME']
name_pollutant_var = POLLUTANTS[pollutant]['CAMS']

for ind, fe in df_fire_events.tail(1).iterrows():
    
    try:
        df_baseline = get_temporal_baseline(
            fe_lat=fe['latitude'],
            fe_long=fe['longitude'],
            timestamp=fe['datetime'],
            days=5,
            pollutant=pollutant,
            fire_mask=True
            # If True, masks fire events from the data used for the calculation of the temporal baseline 
            # slows down processing quite a bit, unfortunately
        )

        plot = create_plot(df_baseline, name_pollutant)
        plot.show()
        
    except Exception as e:
        print(f'Skipping fire {ind} because of the following error: {e}')